In [1]:
import os
import logging
import json
import h2o
import pandas as pd

In [4]:
AZUREML_MODEL_DIR = r"C:\Users\LucaZavarella\OneDrive\MVP\Speaker\202506 - SQL Start\endpoints\model"

In [6]:
model_path = rf"{AZUREML_MODEL_DIR}\MOJO_model.zip"
if os.path.exists(model_path):
    print(f"Model path exists: {model_path}")
else:
    print(f"Model path does not exist: {model_path}")

Model path exists: C:\Users\LucaZavarella\OneDrive\MVP\Speaker\202506 - SQL Start\endpoints\model\MOJO_model.zip


In [7]:
h2o.init()

model = h2o.import_mojo(model_path)

logging.info("Init complete")   

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,18 secs
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.7
H2O_cluster_version_age:,2 months and 15 days
H2O_cluster_name:,LucaZavarella
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.867 Gb
H2O_cluster_total_cores:,14
H2O_cluster_allowed_cores:,14
H2O_cluster_status:,"locked, healthy"


generic Model Build progress: |██████████████████████████████████████████████████| (done) 100%


In [8]:
q = 170.4836

In [9]:
raw_data = """
{
  "data": [
    {
      "Lunghezza": 600,
      "Larghezza": 400,
      "Altezza fiancata": 99,
      "Altezza svaso": 65,
      "TIPOLOGIA SCATOLA": "PL",
      "SUPPORTO": 0,
      "Thickness": 7.5,
      "CodCarta1": 3,
      "CodCarta2": 1,
      "CodCarta3": 1,
      "CodCarta4": 2,
      "CodCarta5": 2,
      "Alt_Testata": 98,
      "C_Ond1": 1.36,
      "C_Ond2": 1.46
    },
    {
      "Lunghezza": 600,
      "Larghezza": 400,
      "Altezza fiancata": 99,
      "Altezza svaso": 55,
      "TIPOLOGIA SCATOLA": "PL",
      "SUPPORTO": 0,
      "Thickness": 7.5,
      "CodCarta1": 3,
      "CodCarta2": 1,
      "CodCarta3": 1,
      "CodCarta4": 2,
      "CodCarta5": 2,
      "Alt_Testata": 98,
      "C_Ond1": 1.36,
      "C_Ond2": 1.46
    }
  ]
}
"""

In [10]:
# Parsing dell'input
data = json.loads(raw_data)
data = data["data"]

# Conversione in DataFrame pandas
input_data = pd.DataFrame(data)

In [11]:
input_data

,Lunghezza,Larghezza,Altezza fiancata,Altezza svaso,TIPOLOGIA SCATOLA,SUPPORTO,Thickness,CodCarta1,CodCarta2,CodCarta3,CodCarta4,CodCarta5,Alt_Testata,C_Ond1,C_Ond2
0,600,400,99,65,PL,0,7.5,3,1,1,2,2,98,1.36,1.46
1,600,400,99,55,PL,0,7.5,3,1,1,2,2,98,1.36,1.46


In [12]:
# Conversione in H2OFrame
input_frame = h2o.H2OFrame(input_data)


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [13]:
logging.info("Making predictions...")

# Effettua la prediction, escludendo la colonna target
prediction = model.predict(input_frame)
prediction_df = prediction.as_data_frame()
prediction_df

generic prediction progress: |

███████████████████████████████████████████████████| (done) 100%


c:\Users\LucaZavarella\miniconda3\envs\bct_ghelfi\Lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


,predict
0,712.442109
1,743.340109


In [14]:
# Estrae la predizione puntuale (si assume che sia nella prima colonna)
prediction_list = prediction_df.iloc[:, 0].tolist()

# Costruisce il JSON di output con intervallo di predizione
results = {"predictions": []}
for p in prediction_list:
    results["predictions"].append({
        "bct_pred": p,
        "bct_lower_90": p - q,
        "bct_upper_90": p + q
    })

logging.info("Request processed")

In [33]:
results

{'predictions': [{'bct_predicted': 712.4421092006138,
   'bct_lower': 542.6326092006138,
   'bct_upper': 882.2516092006138},
  {'bct_predicted': 743.3401089041573,
   'bct_lower': 573.5306089041574,
   'bct_upper': 913.1496089041573}]}